In [2]:
from hdrh.histogram import HdrHistogram
import seaborn as sns
import pandas
import pandas as pd
from matplotlib import pyplot as plt
import os.path
from enum import Enum
import matplotlib as mpl
from typing import *

INVOCATIONS=3

def load_data(folder: str):
    path = os.path.realpath(os.path.expanduser('{}/pauses.csv'.format(folder)))
    if not os.path.isfile(path):
        # print(path)
        return None
    df =  pandas.read_csv(path, names=["nanos"])
    df["millis"] = df["nanos"] / 1000000
    df["micros"] = df["nanos"] / 1000
    return df


def load_and_dump_data(name: str, gc: str, runid: str, bench: str):
    match_fn = lambda x: x.startswith(bench) and (gc in x)
    folders = [ name for name in os.listdir(f'/home/wenyuz/MMTk-Dev/evaluation/results/log/{runid}') if os.path.isdir(f'/home/wenyuz/MMTk-Dev/evaluation/results/log/{runid}/{name}') and match_fn(name) ]
    # print(folders)
    dfs = []
    for folder in folders:
        dfs.append(load_data(f'/home/wenyuz/MMTk-Dev/evaluation/results/log/{runid}/{folder}'))
    df = pd.concat(dfs, ignore_index=True)
    pct = {
        '50': df['millis'].quantile(0.5),
        '95': df['millis'].quantile(0.95),
        '99': df['millis'].quantile(0.99),
        '99.9': df['millis'].quantile(0.999),
        '99.99': df['millis'].quantile(0.9999),
    }
    # return [ bench, percentiles['50'], percentiles['95'] ]
    print(name + ' ' + bench + ':    ' + '  '.join([f'{k}%={x:.1f}' for k, x in pct.items()]))
    x = [pct['50'], pct['99'], pct['99.9'], pct['99.99']]
    print(name + ':    ' + ' & '.join([f'{a:.1f}' for a in x]))

In [3]:
RUNID = 'elk-2022-03-20-Sun-233255'

BENCHMARKS = [
    'avrora', 'batik', 'biojava', 'cassandra', 'fop', 'graphchi', 'h2', 'h2o', 'jython', 'luindex', 'lusearch', 'pmd', 'sunflow', 'tomcat', 'xalan', 'zxing',
]

rows = []
for bm in BENCHMARKS:
    rows.append(load_and_dump_data(gc='.ix.', runid=RUNID, bench=bm))
x = pd.DataFrame(rows)
x.to_csv('lxr-analysis-pause-dist.csv', header=False, index=False)
x

,0,1,2
0,avrora,3.067256,4.567708
1,batik,NaN,NaN
2,biojava,2.379705,4.637500
3,cassandra,10.034609,11.223127
4,fop,5.551724,6.474486
5,graphchi,2.782709,16.713555
6,h2,19.557294,54.498647
7,h2o,20.821624,20.993369
8,jython,5.443290,6.172926
9,luindex,2.418952,2.800999


# 1.5x Table 1

In [7]:
RUNID = 'deer-2022-03-21-Mon-222840'
load_and_dump_data(name='G1', gc='.g1.', runid=RUNID, bench='lusearch')
load_and_dump_data(name='LXR', gc='.ix.', runid=RUNID, bench='lusearch')
load_and_dump_data(name='Shen.', gc='shenandoah.common.hs_perf.latency.dacapochopin', runid=RUNID, bench='lusearch')
load_and_dump_data(name='Shen. 7/8', gc='shenandoah.common.hs_perf.latency.hs_cgc-21.dacapochopin', runid=RUNID, bench='lusearch')

G1 lusearch:    50%=0.5  95%=0.8  99%=0.9  99.9%=1.1  99.99%=1.2
LXR lusearch:    50%=2.2  95%=3.0  99%=3.4  99.9%=9.5  99.99%=11.9
Shen. lusearch:    50%=0.1  95%=0.4  99%=0.4  99.9%=0.7  99.99%=1.5
Shen. 7/8 lusearch:    50%=0.2  95%=0.4  99%=0.6  99.9%=4.6  99.99%=4.9


3x Table 1

In [4]:
RUNID = 'elk-2022-03-20-Sun-233255'
load_and_dump_data(name='G1', gc='.g1.', runid=RUNID, bench='lusearch')
load_and_dump_data(name='LXR', gc='.ix.', runid=RUNID, bench='lusearch')
load_and_dump_data(name='Shen.', gc='shenandoah.common.hs_pauses.dacapochopin', runid=RUNID, bench='lusearch')
load_and_dump_data(name='Shen. 7/8', gc='shenandoah.common.hs_pauses.hs_cgc-21.dacapochopin', runid=RUNID, bench='lusearch')

G1 lusearch:    50%=0.6  95%=0.6  99%=0.7  99.9%=1.1  99.99%=1.1
G1:    0.6 & 0.7 & 1.1 & 1.1
LXR lusearch:    50%=2.3  95%=2.5  99%=3.9  99.9%=11.4  99.99%=13.4
LXR:    2.3 & 3.9 & 11.4 & 13.4
Shen. lusearch:    50%=0.1  95%=0.4  99%=0.5  99.9%=0.6  99.99%=0.7
Shen.:    0.1 & 0.5 & 0.6 & 0.7
Shen. 7/8 lusearch:    50%=0.2  95%=0.5  99%=0.6  99.9%=0.7  99.99%=1.2
Shen. 7/8:    0.2 & 0.6 & 0.7 & 1.2
